In [1192]:
using Pkg
Pkg.activate("CT")
using CT

  Activating project at `~/CT_MPS_mini/CT`


In [1193]:
Pkg.status()

Project CT v0.1.0
Status `~/CT_MPS_mini/CT/Project.toml`
  [c7e460c6] ArgParse v1.2.0
⌃ [6e4b80f9] BenchmarkTools v1.5.0
  [f67ccb44] HDF5 v0.17.2
⌅ [9136182c] ITensors v0.6.16
  [682c06a0] JSON v0.21.4
⌃ [33e6dc65] MKL v0.7.0
⌃ [91a5bcdd] Plots v1.40.5
⌃ [9f0aa9f4] TCIITensorConversion v0.1.4
⌃ [b261b2ec] TensorCrossInterpolation v0.9.10
⌃ [a759f4b9] TimerOutputs v0.5.24
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [1194]:
# benchmark adder_MPO
using ITensors
include("global_adder_passthrough.jl")
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO
using Random
L = 8
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(ram_phy)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
# initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);
# println(initial_state)
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
# @time adder=[adder_MPO(i1,xj,qubit_site,L,phy_ram,phy_list) for i1 in 1:L];
for ram_pos in 1:L
    shift_bit = shift_1_3_bits[ram_phy[ram_pos]]
    println(shift_bit)
end

[1, 2, 3, 4, 5, 6, 7, 8]
[1, 8, 2, 7, 3, 6, 4, 5]
[1, 3, 5, 7, 8, 6, 4, 2]
0
1
1
0
0
1
1
0


In [1195]:
initial_state = productMPS(qubit_site, [1,1,1,1,1,1,1,1])
global_adder(L, qubit_site, shift_1_3_bits, ram_phy, initial_state)

MPS
[1] ((dim=2|id=570|"Qubit,Site,n=1"), (dim=4|id=64|"Link,l=1"))
[2] ((dim=2|id=803|"Qubit,Site,n=2"), (dim=4|id=64|"Link,l=1"), (dim=16|id=938|"Link,l=2"))
[3] ((dim=16|id=938|"Link,l=2"), (dim=2|id=664|"Qubit,Site,n=3"), (dim=16|id=719|"Link,l=3"))
[4] ((dim=16|id=719|"Link,l=3"), (dim=2|id=311|"Qubit,Site,n=4"), (dim=16|id=901|"Link,l=4"))
[5] ((dim=16|id=901|"Link,l=4"), (dim=2|id=458|"Qubit,Site,n=5"), (dim=16|id=91|"Link,l=5"))
[6] ((dim=16|id=91|"Link,l=5"), (dim=2|id=622|"Qubit,Site,n=6"), (dim=16|id=949|"Link,l=6"))
[7] ((dim=16|id=949|"Link,l=6"), (dim=2|id=739|"Qubit,Site,n=7"), (dim=2|id=140|"Link,l=7"))
[8] ((dim=2|id=140|"Link,l=7"), (dim=2|id=450|"Qubit,Site,n=8")')


In [1196]:
psi = contract(initial_state)
println(inds(psi))
print_nonzero_coordinates(psi)

((dim=2|id=570|"Qubit,Site,n=1"), (dim=2|id=803|"Qubit,Site,n=2"), (dim=2|id=664|"Qubit,Site,n=3"), (dim=2|id=311|"Qubit,Site,n=4"), (dim=2|id=458|"Qubit,Site,n=5"), (dim=2|id=622|"Qubit,Site,n=6"), (dim=2|id=739|"Qubit,Site,n=7"), (dim=2|id=450|"Qubit,Site,n=8")')
Nonzero at coordinates (1, 2, 2, 1, 1, 2, 2, 1): 0.9999999999999997
